In [1]:
import pandas as pd
import wandb
import numpy as np

# First run

In [2]:
api_wandb = wandb.Api()

run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/mnipfdm0').scan_history())

In [3]:
run

,_step,progressivity,vkdnw_lambda_1,vkdnw_lambda_5,vkdnw_lambda_8,vkdnw_dim,complexity,vkdnw_lambda_7,arch,trainability,...,vkdnw_lambda_0,vkdnw_chisquare,flops,num_layers,vkdnw_lambda_4,_timestamp,vkdnw_lambda_3,_runtime,model_size,vkdnw_lambda_2
0,0,-Infinity,0.000244,0.194629,7.421612,244,381031544.0,1.810366,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.190722,...,8.016209e-07,190.459459,381031544.0,14,0.068927,1.729607e+09,0.019113,17.287027,3627705.0,0.003603
1,1,-0.017285,0.000006,0.093808,4.455290,248,434595992.0,1.264581,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.186515,...,1.113927e-07,143.468354,434595992.0,14,0.021177,1.729607e+09,0.003287,18.087931,3688617.0,0.000377
2,2,-0.709576,0.000057,0.790009,55.465851,270,402400248.0,10.776753,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.185046,...,9.500461e-06,116.229508,402400248.0,15,0.140649,1.729607e+09,0.024333,18.874209,3652009.0,0.001123
3,3,-0.338296,0.000059,0.523543,13.819300,274,410379800.0,4.165202,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-0.175802,...,2.507468e-06,157.880597,410379800.0,15,0.115631,1.729607e+09,0.014269,19.651100,3679761.0,0.000829
4,4,-0.247345,0.000061,0.276663,11.580867,280,417704712.0,2.290114,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.239618,...,1.729702e-07,176.583333,417704712.0,14,0.097906,1.729607e+09,0.023244,20.503012,3675097.0,0.004763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76246,76246,-0.789663,0.000065,2.150551,226.252106,320,226964764.0,63.217213,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",-0.377860,...,7.038095e-07,120.836364,226964764.0,14,0.445777,1.729693e+09,0.117521,86387.795209,1126625.0,0.009473
76247,76247,-Infinity,0.000005,1.528132,59.838116,274,388668292.0,16.284239,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.268079,...,1.852322e-07,113.655172,388668292.0,13,0.023683,1.729693e+09,0.000678,86388.898207,3500993.0,0.000048
76248,76248,-0.329206,0.000002,2.289304,69.226440,320,396934984.0,13.941657,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.341497,...,1.763321e-07,120.461538,396934984.0,14,0.731657,1.729693e+09,0.137673,86390.667794,3063665.0,0.006313
76249,76249,-Infinity,0.000006,1.378431,40.040604,274,438466012.0,10.738747,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,...",-0.236537,...,9.194015e-07,147.724138,438466012.0,13,0.333114,1.729693e+09,0.069614,86391.824915,3775769.0,0.002764


In [8]:
for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')

In [9]:
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

In [10]:
run.shape

(76251, 28)

In [18]:
run['vkdnw_rank'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True)

In [36]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('az_nas', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
19012,19012.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K5(32,...",449037468.0,322.0,15.0,3685273.0,44.805483,41972.0,21.869541,44.239056,44.209899,44.321068


In [34]:
run.loc[run['vkdnw_dim']==346, ['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
31828,31828.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",449079804.0,346.0,15.0,2313505.0,43.579347,76177.0,22.465845,43.912776,43.73765,44.455743


In [42]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw_rank', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw_rank,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
64775,64775.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",448750524.0,326.0,15.0,3782881.0,44.752932,66546.0,22.283814,44.661257,44.687104,44.652615


In [44]:
#run.loc[run['arch']=='SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,40,2,24,1)SuperResIDWE1K5(40,64,2,48,4)SuperResIDWE1K5(64,192,2,144,5)SuperResIDWE6K5(192,176,2,176,3)SuperConvK1BNRELU(176,2048,1,1)', ]

# Other runs

In [4]:
import pandas as pd
import wandb
import numpy as np

api_wandb = wandb.Api()

In [9]:
# TOP_vkdnw_b_flops480M_layers18
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/2i17bohj').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
86189,86189.0,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K3(48,...",479999294.0,396.0,18.0,3264657.0,45.886779,22.822753,45.730079,45.746149,45.747762


In [8]:
# TOP_vkdnw_b_flops480M_layers15
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/73qqqhqt').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['vkdnw_ratio'] = -(run['vkdnw_lambda_8']/run['vkdnw_lambda_3']).apply(np.log)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw'] = run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run['vkdnw_a'] =  run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_b'] =  run['expressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_c'] =  run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run[['vkdnw_dim', 'vkdnw_ratio']].apply(tuple, axis=1).rank(method='dense', ascending=True).apply(np.log)+run['flops'].rank().apply(np.log)

run[['_step', 'arch', 'flops', 'vkdnw_dim', 'num_layers', 'model_size', 'az_nas', 'vkdnw', 'vkdnw_a', 'vkdnw_b', 'vkdnw_c']].sort_values('vkdnw_b', ascending=False).head(1)

,_step,arch,flops,vkdnw_dim,num_layers,model_size,az_nas,vkdnw,vkdnw_a,vkdnw_b,vkdnw_c
89488,89488.0,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE2K5(80,...",479470486.0,346.0,15.0,2046209.0,45.117084,23.018287,45.124225,45.970446,45.1762


# New

In [ ]:
# TOP_vkdnw_entropy_flops480M_layers14, random params
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/7ov6q9qv').scan_history())

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)

In [19]:
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('rank', ascending=False).head(10)
'SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K3(72,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K5(120,176,1,192,3)SuperResIDWE2K7(176,144,2,168,5)SuperConvK1BNRELU(144,2048,1,1)'

'SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K3(72,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K5(120,176,1,192,3)SuperResIDWE2K7(176,144,2,168,5)SuperConvK1BNRELU(144,2048,1,1)'

In [21]:
# TOP_vkdnw_entropy_flops480M_layers14_2
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('az_nas', ascending=False).head(20)
'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE1K7(120,152,1,240,3)SuperResIDWE6K5(152,112,2,112,5)SuperConvK1BNRELU(112,2048,1,1)'

,_step,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank
40842,40842.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K5(24,...",479792220.0,306.0,44.537932,-0.397331,127.652835,-0.163735,30140
61176,61176.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",479721660.0,310.0,44.500797,-0.396688,126.907147,-0.167757,37962
49181,49181.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.489621,-0.398352,127.517826,-0.169830,34025
47485,47485.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.487284,-0.396580,127.662912,-0.172456,43809
66416,66416.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",479721660.0,310.0,44.483574,-0.395120,126.772448,-0.170604,47624
41941,41941.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.480284,-0.391624,127.548190,-0.173990,56327
54654,54654.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",479690300.0,310.0,44.480045,-0.396493,127.939076,-0.172930,50954
19907,19907.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.474061,-0.395050,127.888495,-0.174318,30852
67007,67007.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479853372.0,310.0,44.473239,-0.397807,127.341166,-0.171584,44537
61232,61232.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",479690300.0,310.0,44.472919,-0.397671,127.784373,-0.172040,54382


In [22]:
# TOP_vkdnw_entropy_flops480M_layers14_3
run[['_step', 'arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank']].sort_values('vkdnw_dim', ascending=False).head(20)
'SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,40,2,24,1)SuperResIDWE1K3(40,96,2,40,1)SuperResIDWE2K5(96,120,2,192,3)SuperResIDWE2K5(120,120,1,192,3)SuperResIDWE4K5(120,88,2,168,5)SuperConvK1BNRELU(88,2048,1,1)'

,_step,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank
17839,17839.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",467638162.0,346.0,41.876212,-0.531196,125.979684,-0.394221,69583
40776,40776.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",450946900.0,346.0,41.356301,-0.481966,131.475983,-0.435285,68067
26163,26163.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",441689918.0,346.0,42.316317,-0.401681,124.427164,-0.354214,64168
32983,32983.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",407537702.0,346.0,41.073718,-0.716819,121.051006,-0.357186,66750
32981,32981.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",466074082.0,346.0,42.709279,-0.400832,125.577932,-0.347882,69329
62544,62544.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",372944878.0,346.0,40.386910,-0.496537,121.454497,-0.384662,65745
47436,47436.0,"SuperConvK3BNRELU(3,8,2,1)SuperResIDWE1K3(8,40...",445578754.0,346.0,42.375321,-0.405643,127.861090,-0.376137,67260
5728,5728.0,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE4K3(32,...",472296494.0,346.0,37.297787,-inf,-inf,-0.323325,64102
5727,5727.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",459772780.0,346.0,39.555881,-0.980489,125.210986,-0.479805,68463
22952,22952.0,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K3(48,...",407410204.0,346.0,40.315074,-0.868098,118.106163,-0.392311,70070


In [2]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/uhoduwim').scan_history()) # VKDNW_EVOLUTION_FLOPS450000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,64,2,40,1)SuperResIDWE2K3(64,96,2,80,2)SuperResIDWE1K3(96,80,2,96,3)SuperResIDWE4K3(80,80,1,56,5)SuperResIDWE1K7(80,112,2,144,1)SuperResIDWE6K5(112,112,1,112,4)SuperConvK1BNRELU(112,2048,1,1)'

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,64,2,40,1)SuperResIDWE2K3(64,96,2,80,2)SuperResIDWE1K3(96,80,2,96,3)SuperResIDWE4K3(80,80,1,56,5)SuperResIDWE1K7(80,112,2,144,1)SuperResIDWE6K5(112,112,1,112,4)SuperConvK1BNRELU(112,2048,1,1)'

In [5]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/il38fg43').scan_history()) # VKDNW_EVOLUTION_FLOPS450000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,40,2,32,1)SuperResIDWE1K5(40,80,2,72,2)SuperResIDWE4K7(80,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
45315,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.849709,-0.071986,128.060867,-0.327774,62288,56.889233
56879,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.826427,-0.070430,127.996855,-0.330729,62258,56.865470
94115,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.886136,-0.072719,128.085451,-0.322321,56732,56.832230
91664,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.797065,-0.080448,128.061676,-0.328387,61211,56.819148
86436,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.761602,-0.081376,127.980293,-0.331026,63391,56.818679
71885,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.791200,-0.070847,127.936033,-0.330552,60578,56.802887
60347,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.751488,-0.080362,127.762275,-0.324682,62968,56.801870
79229,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.822102,-0.079582,128.158788,-0.327177,58317,56.795751
87648,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",449832248.0,274.0,45.775434,-0.080739,127.973894,-0.330540,60849,56.791585
95486,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",449060792.0,274.0,45.752133,-0.081508,127.936150,-0.329528,62010,56.787184


In [6]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/ax1qckwo').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,40,2,40,1)SuperResIDWE1K3(40,96,2,96,2)SuperResIDWE4K7(96,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
88298,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.919450,-0.005563,129.401199,-0.314891,51649,56.771676
95786,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.914185,-0.007558,129.272707,-0.311299,51062,56.754981
60494,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.917586,-0.008506,129.479824,-0.313121,50792,56.753080
84647,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.943584,-0.006968,129.568012,-0.313660,48051,56.723602
72080,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.939005,-0.005929,129.495090,-0.314291,47794,56.713660
88486,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.903008,-0.006568,129.436402,-0.316091,49395,56.710613
92005,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.924352,-0.007248,129.525171,-0.314753,47685,56.696725
84710,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.891349,-0.006715,129.450716,-0.316896,49197,56.694937
75117,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.854507,-0.006092,129.453850,-0.319247,50999,56.694069
62632,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",479716760.0,258.0,45.839231,-0.008846,129.354128,-0.316961,51246,56.683624


In [16]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/592dqlhj').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)
'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,40,2,40,1)SuperResIDWE1K3(40,56,2,64,5)SuperResIDWE4K7(56,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
55557,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.873608,-0.111259,150.575394,-0.274803,58976,56.858494
83128,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.878883,-0.113637,150.628929,-0.273752,58251,56.851400
72361,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.865552,-0.110750,150.603093,-0.276053,58824,56.847857
18241,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.840663,-0.116069,150.590804,-0.276115,60300,56.847750
96089,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.851303,-0.116572,150.607646,-0.274938,59516,56.845303
40719,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.863732,-0.114722,150.522379,-0.272747,58595,56.842137
82043,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.835400,-0.111546,150.461509,-0.276235,60182,56.840528
96252,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.840100,-0.112792,150.693312,-0.277675,59794,56.838761
25684,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.849746,-0.111478,150.439522,-0.274094,59091,56.836580
46796,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",479792024.0,348.0,45.852149,-0.114584,150.493744,-0.274009,58829,56.834539


In [5]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/47mm5i5f').scan_history()) # VKDNW_EVOLUTION_FLOPS520000000_LAYERS14_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run.loc[run['flops']<480000000, ['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,40,2,32,1)SuperResIDWE2K7(40,72,2,64,2)SuperResIDWE4K7(72,96,2,96,5)SuperResIDWE2K7(96,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)' # TOP

'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,56,2,48,2)SuperResIDWE1K3(56,96,2,128,1)SuperResIDWE2K5(96,112,2,120,3)SuperResIDWE4K5(112,112,2,112,3)SuperResIDWE4K5(112,112,1,112,4)SuperConvK1BNRELU(112,2048,1,1)' # TOP 4

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
21145,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",479114648.0,274.0,42.120346,-0.092956,127.517635,-0.327002,26983,52.323308
36874,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",472030424.0,274.0,42.061262,-0.085270,128.225694,-0.332450,28057,52.303256
36691,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",466310360.0,274.0,42.035831,-0.075512,128.407858,-0.332190,28774,52.303058
14529,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,...",471643716.0,298.0,41.711740,-0.161149,123.448049,-0.317673,36767,52.224096
40971,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",468718808.0,274.0,42.030214,-0.096070,128.422935,-0.332836,26549,52.216962
11288,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",459416158.0,310.0,41.561901,-0.206044,125.708663,-0.321939,41891,52.204727
21220,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",469622662.0,310.0,41.523937,-0.351454,125.942455,-0.321544,42493,52.181031
10212,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,...",471643716.0,298.0,41.642727,-0.171950,123.668597,-0.325461,37456,52.173649
45931,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K5(32,...",457157552.0,274.0,42.001464,-0.094357,129.750493,-0.331660,26098,52.171078
19605,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K5(24,...",475963654.0,310.0,41.547360,-0.343593,125.096295,-0.323049,40357,52.152880


In [6]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/u6n06lkq').scan_history()) # VKDNW_EVOLUTION_FLOPS520000000_LAYERS17_PARAMS_RANDOM_FIXED

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run.loc[run['flops']<480000000, ['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,24,2,32,3)SuperResIDWE4K7(24,40,2,48,1)SuperResIDWE2K7(40,120,2,96,3)SuperResIDWE2K7(120,120,1,96,4)SuperResIDWE2K7(120,224,2,224,5)SuperConvK1BNRELU(224,2048,1,1)' # TOP

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
25592,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",475734040.0,388.0,40.658166,-0.262953,149.075095,-0.335206,29634,50.954843
27879,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K5(96,...",471493580.0,388.0,40.633798,-0.359137,144.746202,-0.290686,29485,50.925435
27470,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...",475948660.0,392.0,40.600576,-0.405251,142.360233,-0.279160,30362,50.921523
19630,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K7(96,...",479188638.0,392.0,40.587346,-0.574870,145.356719,-0.279303,30457,50.911417
33746,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",473143704.0,364.0,40.923801,-0.213260,154.081091,-0.315414,21498,50.899516
13911,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",471293072.0,388.0,40.654686,-0.207170,151.711402,-0.344474,27696,50.883729
24365,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",468827000.0,384.0,40.685900,-0.239196,152.438479,-0.336260,26600,50.874567
27121,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",475453368.0,380.0,40.739191,-0.274071,150.902516,-0.332096,24678,50.852858
24401,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K7(80,...",468827000.0,384.0,40.667473,-0.252667,152.850837,-0.337892,26253,50.843008
22121,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K5(64,...",475545880.0,388.0,40.540935,-0.359168,148.225889,-0.335255,29774,50.842326


In [1]:
import wandb
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/ax1qckwo').scan_history()) # VKDNW_EVOLUTION_FLOPS480000000_LAYERS14_PARAMS_RANDOM_FIXED

In [2]:
run.head(50)

,flops,vkdnw_lambda_1,vkdnw_dim,vkdnw_lambda_6,vkdnw_lambda_8,arch,trainability,vkdnw_entropy,vkdnw_lambda_3,vkdnw_lambda_2,...,vkdnw_lambda_5,_timestamp,vkdnw_lambda_4,vkdnw_lambda_7,vkdnw_chisquare,num_layers,_step,expressivity,progressivity,vkdnw_lambda_0
0,450508056.0,2.999272e-08,244,0.025921,0.552194,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",-0.349545,0.027253,0.000035,1.327641e-06,...,0.006757,1.730193e+09,0.000305,0.107971,152.429907,14,0,119.783098,-0.010152,0.000000e+00
1,457582872.0,0.000000e+00,244,0.003351,0.193395,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",-0.350593,0.008466,0.000002,3.899601e-07,...,0.000456,1.730193e+09,0.000022,0.016570,140.450450,14,1,119.977228,-0.013572,0.000000e+00
2,457582872.0,2.783879e-07,244,0.019030,0.606208,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",-0.352773,0.010086,0.000052,2.409316e-06,...,0.003320,1.730193e+09,0.000744,0.109983,156.056604,14,2,120.039291,-0.012472,0.000000e+00
3,477401804.0,1.169755e-08,306,0.252917,9.226439,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.585437,0.020041,0.000122,3.961710e-06,...,0.049668,1.730193e+09,0.003580,1.967010,132.494118,15,3,-Infinity,-Infinity,0.000000e+00
4,381031544.0,9.205151e-08,244,0.002528,0.087383,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.366132,0.034717,0.000009,6.640306e-07,...,0.000551,1.730193e+09,0.000124,0.017681,161.179487,14,4,117.774245,-0.507675,0.000000e+00
5,434595992.0,1.524530e-07,248,0.003201,0.080045,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.357056,0.007461,0.000009,1.924249e-06,...,0.000723,1.730193e+09,0.000069,0.010745,169.646018,14,5,118.819937,-0.009373,0.000000e+00
6,402400248.0,0.000000e+00,270,0.015578,0.893442,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.351708,0.005039,0.000005,6.091686e-07,...,0.001244,1.730193e+09,0.000054,0.162234,130.349515,15,6,124.559421,-0.722001,0.000000e+00
7,410379800.0,1.011276e-06,274,0.026810,1.205330,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-0.354069,0.004473,0.000083,2.054571e-05,...,0.006652,1.730193e+09,0.001018,0.129387,167.821782,15,7,123.300824,-0.361047,0.000000e+00
8,417704712.0,2.186542e-07,280,0.027110,0.600316,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.449067,0.027124,0.000035,1.698244e-06,...,0.004529,1.730193e+09,0.000893,0.140381,174.209524,14,8,123.344525,-0.250491,0.000000e+00
9,436358424.0,2.217269e-07,244,0.050268,0.602638,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...",-0.360748,0.006864,0.000031,1.029203e-06,...,0.008072,1.730193e+09,0.001152,0.130753,173.264151,14,9,119.394996,-0.00894,0.000000e+00


In [6]:
run['arch']

0        SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
1        SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...
2        SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...
3        SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...
4        SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K7(16,...
                               ...                        
99995    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
99996    SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...
99997    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
99998    SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...
99999    SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...
Name: arch, Length: 100000, dtype: object

In [7]:
import re
def remove_parentheses(text):
    # Regular expression pattern to match any substring within parentheses
    return re.sub(r"\(.*?\)", "/", text)

In [8]:
list_all = []
for arch in run.arch:
    list_add = remove_parentheses(arch).split('/')
    list_all = list_all + list_add
list_all = set(list_all)

In [9]:
set(list_all)

{'',
 'SuperConvK1BNRELU',
 'SuperConvK3BNRELU',
 'SuperResIDWE1K3',
 'SuperResIDWE1K5',
 'SuperResIDWE1K7',
 'SuperResIDWE2K3',
 'SuperResIDWE2K5',
 'SuperResIDWE2K7',
 'SuperResIDWE4K3',
 'SuperResIDWE4K5',
 'SuperResIDWE4K7',
 'SuperResIDWE6K3',
 'SuperResIDWE6K5',
 'SuperResIDWE6K7'}

In [10]:
(run['arch']).sort_values()

10085    SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K3(10...
7799     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K3(10...
2424     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE1K7(10...
3013     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE4K3(10...
1420     SuperConvK3BNRELU(3,104,2,1)SuperResIDWE4K3(10...
                               ...                        
2179     SuperConvK3BNRELU(3,96,2,1)SuperResIDWE4K5(96,...
21010    SuperConvK3BNRELU(3,96,2,1)SuperResIDWE4K7(96,...
1877     SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
970      SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
47024    SuperConvK3BNRELU(3,96,2,1)SuperResIDWE6K3(96,...
Name: arch, Length: 100000, dtype: object

# Feasible

In [4]:
import wandb
import pandas as pd
api_wandb = wandb.Api()

In [5]:
runs_feasible = None
for run_str in [
    'nazderaze/VKDNW/le027c6i',
    'nazderaze/VKDNW/e5ghgrna',
    'nazderaze/VKDNW/9t4z9soy',
    'nazderaze/VKDNW/xu12jwh3',
    'nazderaze/VKDNW/p4w49gbw',
    'nazderaze/VKDNW/8o8nv1bw',
    'nazderaze/VKDNW/a2ohrrxp',
    'nazderaze/VKDNW/uk275wky',
    'nazderaze/VKDNW/5v9c0ueh',
    'nazderaze/VKDNW/vt926kqu',
]:
    run = pd.DataFrame(api_wandb.run(run_str).scan_history())
    
    if runs_feasible is None:
        runs_feasible = run
    else:
        runs_feasible = pd.concat([runs_feasible, run], ignore_index=True)
runs_feasible.shape

(106672, 9)

In [8]:
runs_feasible[['flops', 'layers', 'net_str']].drop_duplicates()

,flops,layers,net_str
0,4.345053e+08,11,"SuperConvK3BNRELU(3,21,2,1)SuperResK1K3K1(21,3..."
1,4.136010e+08,7,"SuperConvK3BNRELU(3,8,2,1)SuperResIDWE6K5(8,21..."
2,4.351690e+08,13,"SuperConvK3BNRELU(3,15,2,1)SuperResK5K5(15,39,..."
3,4.531786e+08,15,"SuperConvK3BNRELU(3,8,2,1)SuperResK5K5(8,12,2,..."
4,4.253229e+08,11,"SuperConvK3BNRELU(3,11,2,1)SuperResIDWE4K5(11,..."
...,...,...,...
106667,4.734557e+08,12,"SuperConvK3BNRELU(3,17,2,1)SuperResK5K5(17,31,..."
106668,4.590909e+08,13,"SuperConvK3BNRELU(3,18,2,1)SuperResK5K5(18,27,..."
106669,4.891607e+08,9,"SuperConvK3BNRELU(3,8,2,1)SuperResIDWE4K3(8,26..."
106670,4.370465e+08,10,"SuperConvK3BNRELU(3,24,2,1)SuperResK5K5(24,44,..."


# Feasible evaluated

In [5]:
import wandb
import pandas as pd
import numpy as np

api_wandb = wandb.Api()

runs_feasible = None
for run_str in [
    'nazderaze/VKDNW/8yvlu0hz',
    'nazderaze/VKDNW/m4pkljka',
    'nazderaze/VKDNW/d6a4vr97',
    'nazderaze/VKDNW/841g5w2y',
    'nazderaze/VKDNW/nvu9m5x1',
    'nazderaze/VKDNW/t6gk34am',
    'nazderaze/VKDNW/2ezsk73r',
    'nazderaze/VKDNW/sxkpoqsc',
    'nazderaze/VKDNW/ejlihlxn',
    'nazderaze/VKDNW/3z8jirir',
    'nazderaze/VKDNW/x2we5ioq',
    'nazderaze/VKDNW/twbjv2r5',
]:
    run = pd.DataFrame(api_wandb.run(run_str).scan_history())
    
    if runs_feasible is None:
        runs_feasible = run
    else:
        runs_feasible = pd.concat([runs_feasible, run], ignore_index=True)
runs_feasible.shape

(80761, 20)

In [6]:
for col in run.columns:
    if col != 'arch':
        runs_feasible[col] = runs_feasible[col].astype('float')
        
runs_feasible['rank'] = runs_feasible[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
runs_feasible['vkdnw_rank'] = runs_feasible['rank'].rank().apply(np.log)+runs_feasible['flops'].rank().apply(np.log)

runs_feasible.sort_values('vkdnw_rank').head(20)

,_timestamp,vkdnw_lambda_5,vkdnw_lambda_4,vkdnw_lambda_10,vkdnw_lambda_0,vkdnw_lambda_6,model_size,_step,num_layers,flops,...,net_str,_runtime,vkdnw_lambda_2,vkdnw_dim,vkdnw_lambda_7,vkdnw_lambda_1,vkdnw_lambda_8,vkdnw_lambda_9,rank,vkdnw_rank
48331,1.730739e+09,0.000398,9.616578e-05,26.241833,4.696227e-11,0.001577,3766539.0,5920,10,4.050005e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResK5K5(9,30,2,...",2594.871201,1.056092e-06,160,0.006318,8.546944e-09,0.031754,0.242752,29661,10.297622
59116,1.730740e+09,0.000114,2.463648e-05,22.748608,1.634594e-10,0.000497,6299926.0,7026,8,4.050425e+08,...,"SuperConvK3BNRELU(3,24,2,1)SuperResK5K5(24,24,...",2887.997691,2.724802e-07,128,0.002013,4.585553e-09,0.012837,0.099357,913,10.480298
1412,1.730738e+09,0.000543,4.875342e-05,353.523895,0.000000e+00,0.003538,4965396.0,1412,11,4.050112e+08,...,"SuperConvK3BNRELU(3,10,2,1)SuperResIDWE6K5(10,...",624.930468,5.245573e-08,134,0.016310,0.000000e+00,0.071369,0.435127,5761,10.604776
8852,1.730738e+09,0.000933,1.641957e-04,362.794861,0.000000e+00,0.004221,10736498.0,1722,10,4.050015e+08,...,"SuperConvK3BNRELU(3,11,2,1)SuperResIDWE2K3(11,...",765.326662,1.346991e-06,152,0.022200,1.867706e-07,0.093850,0.522154,21749,10.680470
52049,1.730741e+09,0.000059,9.675537e-06,196.141449,0.000000e+00,0.000480,15413090.0,9638,10,4.055845e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResK3K3(9,23,2,...",4087.085303,1.791661e-07,128,0.003403,4.193310e-08,0.019926,0.150105,98,10.855956
14821,1.730738e+09,0.000056,1.328129e-05,3.785370,1.308198e-11,0.000173,23523303.0,656,8,4.050834e+08,...,"SuperConvK3BNRELU(3,14,2,1)SuperResIDWE4K5(14,...",290.672760,5.222385e-08,128,0.000439,3.870144e-10,0.001382,0.011450,645,10.863699
2156,1.730738e+09,0.000073,4.180137e-06,2837.166260,0.000000e+00,0.000541,23843086.0,2156,10,4.161864e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResIDWE1K3(9,13...",935.005019,0.000000e+00,128,0.010757,0.000000e+00,0.049175,0.345076,6,11.063383
7110,1.730740e+09,0.001578,3.361582e-04,10045.578125,0.000000e+00,0.007265,10366727.0,7110,10,4.760278e+08,...,"SuperConvK3BNRELU(3,20,2,1)SuperResIDWE1K3(20,...",2976.267987,3.323111e-06,128,0.027276,7.839443e-07,0.116389,0.732462,1,11.070879
17785,1.730739e+09,0.000017,5.766090e-07,50.611065,0.000000e+00,0.000113,14525713.0,3620,9,4.050121e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResK3K3(9,23,2,...",1942.950561,7.053854e-09,138,0.001752,1.377187e-10,0.010943,0.110678,9558,11.244575
77148,1.730740e+09,0.000079,9.251617e-06,16.339138,0.000000e+00,0.000456,3372231.0,5873,8,4.051594e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResIDWE6K5(9,33...",3398.198494,4.195666e-08,128,0.001773,2.347106e-09,0.007199,0.044458,596,11.407521


In [8]:
runs_feasible.loc[runs_feasible['model_size']<2700000, ].sort_values('vkdnw_rank')

,_timestamp,vkdnw_lambda_5,vkdnw_lambda_4,vkdnw_lambda_10,vkdnw_lambda_0,vkdnw_lambda_6,model_size,_step,num_layers,flops,...,net_str,_runtime,vkdnw_lambda_2,vkdnw_dim,vkdnw_lambda_7,vkdnw_lambda_1,vkdnw_lambda_8,vkdnw_lambda_9,rank,vkdnw_rank
65620,1.730739e+09,0.002517,0.000663,9.046603e+01,0.0,0.007206,2233783.0,4037,10,4.053462e+08,...,"SuperConvK3BNRELU(3,13,2,1)SuperResK3K3(13,44,...",1937.784093,7.528184e-07,128,0.021149,1.416080e-08,0.070112,0.280599,802,12.483166
41613,1.730741e+09,0.000106,0.000019,1.653275e+02,0.0,0.000420,2239074.0,8557,10,4.124372e+08,...,"SuperConvK3BNRELU(3,20,2,1)SuperResIDWE6K3(20,...",4500.327448,1.053526e-07,128,0.002010,2.932127e-08,0.010005,0.068171,41,12.573362
55315,1.730738e+09,0.000285,0.000012,1.200516e+02,0.0,0.001934,1772991.0,3225,10,4.084561e+08,...,"SuperConvK3BNRELU(3,9,2,1)SuperResK1K5K1(9,24,...",1353.671429,6.773789e-08,128,0.008186,4.956423e-09,0.053403,0.322891,569,14.452203
43266,1.730737e+09,0.000888,0.000113,2.115906e+01,0.0,0.003732,1777923.0,855,10,4.064127e+08,...,"SuperConvK3BNRELU(3,24,2,1)SuperResK1K5K1(24,3...",512.325131,5.525994e-08,128,0.012468,8.108651e-10,0.039712,0.224069,1820,14.675172
40023,1.730740e+09,0.000301,0.000047,1.425708e+01,0.0,0.001254,2228410.0,6967,8,4.052375e+08,...,"SuperConvK3BNRELU(3,15,2,1)SuperResK1K5K1(15,4...",3609.459325,5.591011e-07,140,0.004470,1.391421e-09,0.019160,0.076406,11596,14.756578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37469,1.730739e+09,0.326901,0.027311,1.355409e+04,0.0,1.416555,2692072.0,4413,16,4.940955e+08,...,"SuperConvK3BNRELU(3,24,2,1)SuperResK1K3K1(24,3...",2169.864195,1.241839e-05,268,5.193804,1.861180e-06,19.219801,81.431137,76695,22.537349
32484,1.730739e+09,1.868636,0.141060,1.545539e+06,0.0,38.965538,2677391.0,3990,16,4.901067e+08,...,"SuperConvK3BNRELU(3,22,2,1)SuperResK1K3K1(22,2...",1661.659142,1.423244e-04,360,495.569885,3.056032e-06,3213.730713,14970.749023,80614,22.543375
60334,1.730740e+09,32.614403,3.033341,2.720827e+05,0.0,196.037338,2308631.0,8244,17,4.910159e+08,...,"SuperConvK3BNRELU(3,23,2,1)SuperResK1K3K1(23,4...",3381.217000,1.080168e-04,358,786.829712,7.417043e-06,3251.848633,20257.669922,80606,22.553346
45206,1.730738e+09,0.156795,0.021010,1.310983e+03,0.0,0.498578,2402696.0,2795,14,4.946190e+08,...,"SuperConvK3BNRELU(3,20,2,1)SuperResIDWE1K5(20,...",1314.804600,2.770879e-05,284,1.630466,6.451955e-07,6.875823,45.417728,78337,22.563819


In [7]:
import numpy as np

In [ ]:
'SuperConvK3BNRELU(3,15,2,1)SuperResK1K3K1(15,27,2,24,2)SuperResIDWE4K5(27,46,2,28,1)SuperResK1K3K1(46,151,2,138,1)SuperResK3K3(151,186,2,151,1)SuperResK5K5(186,386,2,344,3)SuperConvK1BNRELU(386,2048,1,1)'

'SuperConvK3BNRELU(3,8,2,1)SuperResK1K5K1(8,29,2,10,2)SuperResIDWE6K5(29,93,2,63,1)SuperResK1K5K1(93,99,2,93,1)SuperResK3K3(99,306,2,188,1)SuperResK3K3(306,430,2,364,3)SuperConvK1BNRELU(430,2048,1,1)'

'SuperConvK3BNRELU(3,9,2,1)SuperResK3K3(9,23,2,15,1)SuperResK1K3K1(23,56,2,30,1)SuperResK3K3(56,153,2,126,1)SuperResIDWE6K3(153,303,2,166,3)SuperResK3K3(303,849,2,777,1)SuperConvK1BNRELU(849,2048,1,1)'

('SuperConvK3BNRELU(3,18,2,1)SuperResK1K3K1(18,29,2,25,2)SuperResIDWE1K5(29,41,2,29,1)SuperResK3K3(41,137,2,69,2)SuperResK3K3(137,280,2,196,3)SuperResK5K5(280,364,2,304,2)SuperConvK1BNRELU(364,2048,1,1)'
 '')

# SMAL MODELS
'SuperConvK3BNRELU(3,23,2,1)SuperResK1K3K1(23,41,2,31,5)SuperResK1K3K1(41,44,2,42,1)SuperResIDWE6K3(44,56,2,48,3)SuperResIDWE1K3(56,160,2,139,3)SuperResIDWE2K5(160,198,2,179,3)SuperConvK1BNRELU(198,2048,1,1)'

'SuperConvK3BNRELU(3,20,2,1)SuperResIDWE1K5(20,31,2,23,5)SuperResK1K5K1(31,85,2,45,3)SuperResK1K5K1(85,87,2,85,1)SuperResIDWE4K3(87,93,2,93,2)SuperResIDWE2K5(93,343,2,197,1)SuperConvK1BNRELU(343,2048,1,1)'

'SuperConvK3BNRELU(3,14,2,1)SuperResK1K5K1(14,14,2,14,5)SuperResK1K5K1(14,53,2,31,3)SuperResK1K5K1(53,56,2,56,3)SuperResIDWE1K5(56,78,2,62,1)SuperResIDWE2K5(78,236,2,101,1)SuperConvK1BNRELU(236,2048,1,1)'

'SuperConvK3BNRELU(3,9,2,1)SuperResK1K5K1(9,28,2,12,1)SuperResIDWE4K5(28,86,2,50,3)SuperResK1K5K1(86,89,2,89,2)SuperResIDWE2K3(89,102,2,90,3)SuperResK1K5K1(102,163,2,117,1)SuperConvK1BNRELU(163,2048,1,1)'

'SuperConvK3BNRELU(3,13,2,1)SuperResK5K5(13,47,2,14,1)SuperResK1K5K1(47,64,2,52,3)SuperResIDWE1K5(64,78,2,73,2)SuperResIDWE2K5(78,112,2,100,5)SuperResIDWE1K3(112,131,2,112,2)SuperConvK1BNRELU(131,2048,1,1)'

In [5]:
runs_feasible.loc[runs_feasible['net_str']=='SuperConvK3BNRELU(3,15,2,1)SuperResK1K3K1(15,27,2,24,2)SuperResIDWE4K5(27,46,2,28,1)SuperResK1K3K1(46,151,2,138,1)SuperResK3K3(151,186,2,151,1)SuperResK5K5(186,386,2,344,3)SuperConvK1BNRELU(386,2048,1,1)', ]

,_timestamp,vkdnw_lambda_5,vkdnw_lambda_4,vkdnw_lambda_10,vkdnw_lambda_0,vkdnw_lambda_6,model_size,_step,num_layers,flops,...,net_str,_runtime,vkdnw_lambda_2,vkdnw_dim,vkdnw_lambda_7,vkdnw_lambda_1,vkdnw_lambda_8,vkdnw_lambda_9,rank,vkdnw_rank
77282,1.730740e+09,0.000091,0.000015,9.742836,0.0,0.000766,20047970.0,6007,10,405076096.5,...,"SuperConvK3BNRELU(3,15,2,1)SuperResK1K3K1(15,2...",3469.362211,4.724164e-08,128,0.003988,1.090296e-09,0.026536,0.124472,1949,11.823567


# Exhaustive final

In [11]:
import wandb
import pandas as pd
import numpy as np

api_wandb = wandb.Api() # large models (without VKDNW)

runs_feasible = None
for run_str in [
    'nazderaze/VKDNW/fs3hu56a',
    'nazderaze/VKDNW/cchmltz1',
    'nazderaze/VKDNW/urzaw444',
    'nazderaze/VKDNW/uxlvemd7'
]:
    run = pd.DataFrame(api_wandb.run(run_str).scan_history())
    
    if runs_feasible is None:
        runs_feasible = run
    else:
        runs_feasible = pd.concat([runs_feasible, run], ignore_index=True)
runs_feasible.shape

(180385, 10)

In [12]:
for col in run.columns:
    if col != 'arch':
        runs_feasible[col] = runs_feasible[col].astype('float')
        
runs_feasible['vkdnw_rank'] = runs_feasible['flops'].rank().apply(np.log)+runs_feasible['expressivity'].rank().apply(np.log)+runs_feasible['progressivity'].rank().apply(np.log)

runs_feasible.sort_values('vkdnw_rank')

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,48,2,32,1)SuperResIDWE1K7(48,80,2,64,1)SuperResIDWE2K3(80,192,2,176,1)SuperResIDWE6K3(192,184,2,192,4)SuperResIDWE1K7(184,616,2,408,5)SuperConvK1BNRELU(616,2048,1,1)' # VKDNW_BRUTE_4

,complexity,expressivity,_step,arch,progressivity,_runtime,num_layers,flops,model_size,_timestamp,vkdnw_rank
117,60064738.0,87.432874,117.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K5(16,...",-1.098723,75.231224,13.0,60064738.0,1009369.0,1.730761e+09,19.466754
149,74443934.0,79.955724,149.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-1.586276,84.750038,12.0,74443934.0,2069437.0,1.730761e+09,19.510509
735,105302806.0,-inf,735.0,"SuperConvK3BNRELU(3,8,2,1)SuperResIDWE1K3(8,16...",-inf,373.036951,14.0,105302806.0,4726769.0,1.730761e+09,19.571451
245,107262958.0,-inf,245.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE1K3(16,...",-inf,136.738319,13.0,107262958.0,2116377.0,1.730761e+09,19.731793
43757,109243548.0,-inf,1735.0,"SuperConvK3BNRELU(3,16,2,1)SuperResIDWE2K5(16,...",-inf,570.892756,11.0,109243548.0,655593.0,1.730761e+09,19.901692
...,...,...,...,...,...,...,...,...,...,...,...
39894,449978300.0,124.146266,39894.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.093651,35608.230288,14.0,449978300.0,13081345.0,1.730796e+09,36.285769
40526,449978300.0,124.189848,40526.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.093681,36321.092204,14.0,449978300.0,13081345.0,1.730797e+09,36.285989
41611,449978300.0,124.127547,41611.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.093089,37314.978622,14.0,449978300.0,13081345.0,1.730798e+09,36.286081
36388,449978300.0,124.113703,36388.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K5(24,...",-0.091368,31715.988699,14.0,449978300.0,13081345.0,1.730792e+09,36.286831


In [13]:
runs_feasible.columns

Index(['complexity', 'expressivity', '_step', 'arch', 'progressivity',
       '_runtime', 'num_layers', 'flops', 'model_size', '_timestamp',
       'vkdnw_rank'],
      dtype='object')

In [14]:
import wandb
import pandas as pd
import numpy as np

api_wandb = wandb.Api() # small models (with VKDNW)

runs_feasible = None
for run_str in [
    'nazderaze/VKDNW/fhahg0mz',
    'nazderaze/VKDNW/dub0g89h',
    'nazderaze/VKDNW/m68lu6zb',
    'nazderaze/VKDNW/aq05vf54'
]:
    run = pd.DataFrame(api_wandb.run(run_str).scan_history())
    
    if runs_feasible is None:
        runs_feasible = run
    else:
        runs_feasible = pd.concat([runs_feasible, run], ignore_index=True)
runs_feasible.shape

(177097, 24)

In [15]:
for col in run.columns:
    if col != 'arch':
        runs_feasible[col] = runs_feasible[col].astype('float')
        
runs_feasible['rank'] = runs_feasible[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
runs_feasible['vkdnw_rank'] = runs_feasible['rank'].rank().apply(np.log)+runs_feasible['flops'].rank().apply(np.log)+runs_feasible['expressivity'].rank().apply(np.log)+runs_feasible['progressivity'].rank().apply(np.log)

runs_feasible.sort_values('vkdnw_rank')
'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,48,2,24,1)SuperResIDWE4K3(48,96,2,80,1)SuperResIDWE1K5(96,96,2,112,1)SuperResIDWE1K5(96,288,2,192,5)SuperResIDWE2K5(288,352,2,736,3)SuperResIDWE6K3(352,688,1,616,1)SuperConvK1BNRELU(688,2048,1,1)' # VKDNW_BRUTE_1
'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,64,2,48,1)SuperResIDWE6K3(64,96,2,72,1)SuperResIDWE1K5(96,78,2,73,2)SuperResIDWE2K7(78,216,2,152,5)SuperResIDWE2K3(216,528,2,352,3)SuperConvK1BNRELU(528,2048,1,1)' # VKDNW_BRUTE_2
'SuperConvK3BNRELU(3,128,2,1)SuperResIDWE2K3(128,56,2,24,2)SuperResIDWE4K3(56,96,2,80,1)SuperResIDWE2K3(96,80,2,88,1)SuperResIDWE6K5(80,56,2,96,5)SuperResIDWE4K5(56,328,2,144,3)SuperConvK1BNRELU(328,2048,1,1)' # VKDNW_BRUTE_3


,complexity,vkdnw_lambda_1,expressivity,_step,arch,progressivity,vkdnw_lambda_2,vkdnw_lambda_3,vkdnw_lambda_4,vkdnw_lambda_9,...,flops,vkdnw_lambda_0,vkdnw_lambda_5,vkdnw_lambda_6,model_size,vkdnw_entropy,_timestamp,vkdnw_dim,rank,vkdnw_rank
76623,127067050.0,6.955986e-10,-inf,503.0,"SuperConvK3BNRELU(3,8,2,1)SuperResK1K5K1(8,28,...",-inf,6.152391e-09,0.000002,0.000020,0.178008,...,127067050.0,0.0,0.000148,0.000645,625165.0,0.063864,1.730762e+09,130.0,1,23.506341
76582,83979428.0,2.844745e-09,-inf,462.0,"SuperConvK3BNRELU(3,8,2,1)SuperResK1K5K1(8,28,...",-inf,3.711278e-08,0.000018,0.000143,0.564550,...,83979428.0,0.0,0.000818,0.003328,559685.0,0.139669,1.730762e+09,156.0,18,23.610580
76655,60010676.0,7.264537e-10,51.036385,535.0,"SuperConvK3BNRELU(3,9,2,1)SuperResK1K5K1(9,28,...",-1.006227,7.836885e-08,0.000003,0.000025,0.118640,...,60010676.0,0.0,0.000195,0.000960,424625.0,0.145828,1.730762e+09,156.0,19,23.797421
40285,48186301.5,3.494212e-10,57.475944,4022.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K5(24,...",-0.945848,1.682698e-08,0.000005,0.000048,0.125112,...,48186301.5,0.0,0.000281,0.001126,875381.0,0.121561,1.730765e+09,196.0,530,25.894564
76999,83134060.0,1.880414e-09,-inf,879.0,"SuperConvK3BNRELU(3,24,2,1)SuperResK1K5K1(24,2...",-inf,2.115620e-07,0.000022,0.000173,0.794376,...,83134060.0,0.0,0.000887,0.004265,700381.0,0.182044,1.730762e+09,186.0,223,26.051872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167127,476006040.0,3.135631e-08,124.279241,43043.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",-0.292181,4.529919e-07,0.000006,0.000192,8.523443,...,476006040.0,0.0,0.004052,0.030266,23910169.0,0.189478,1.730791e+09,320.0,175294,48.189349
157675,476006040.0,2.677582e-08,124.335995,33591.0,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K3(24,...",-0.298758,9.693409e-07,0.000045,0.000974,8.115309,...,476006040.0,0.0,0.009262,0.062439,23910169.0,0.247304,1.730784e+09,320.0,176621,48.190187
163678,476076231.0,4.421062e-09,120.720838,39594.0,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",-0.288360,3.429166e-07,0.000025,0.000522,8.806160,...,476076231.0,0.0,0.003929,0.028529,8690599.0,0.239035,1.730788e+09,320.0,176522,48.193758
168965,477715652.0,5.690438e-08,123.888409,44881.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.292270,1.209994e-06,0.000173,0.001568,11.090195,...,477715652.0,0.0,0.011826,0.062884,21038105.0,0.196289,1.730792e+09,320.0,175553,48.197440


In [23]:
runs_feasible.loc[runs_feasible['arch']=='SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,48,2,24,1)SuperResIDWE4K3(48,96,2,80,1)SuperResIDWE1K5(96,96,2,112,1)SuperResIDWE1K5(96,288,2,192,5)SuperResIDWE2K5(288,352,2,736,3)SuperResIDWE6K3(352,688,1,616,1)SuperConvK1BNRELU(688,2048,1,1)', ]

,complexity,vkdnw_lambda_1,expressivity,_step,arch,progressivity,vkdnw_lambda_2,vkdnw_lambda_3,vkdnw_lambda_4,vkdnw_lambda_9,...,flops,vkdnw_lambda_0,vkdnw_lambda_5,vkdnw_lambda_6,model_size,vkdnw_entropy,_timestamp,vkdnw_dim,rank,vkdnw_rank
162145,479234728.0,1.083534e-08,124.543492,38061.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.326921,9.344146e-07,0.000031,0.000773,4.073236,...,479234728.0,0.0,0.004371,0.015750,21728825.0,0.230817,1.730787e+09,320.0,176399,48.182822
164628,479234728.0,0.000000e+00,124.604450,40544.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.290880,1.971001e-06,0.000008,0.000171,10.949025,...,479234728.0,0.0,0.002900,0.016859,21728825.0,0.009355,1.730789e+09,320.0,146955,48.034955
166569,479234728.0,3.121487e-07,124.531677,42485.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.332009,1.923260e-06,0.000031,0.000944,12.350678,...,479234728.0,0.0,0.008887,0.043794,21728825.0,0.050290,1.730791e+09,320.0,156119,48.052945
166570,479234728.0,0.000000e+00,124.709987,42486.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.299439,1.012158e-07,0.000009,0.000451,13.332430,...,479234728.0,0.0,0.002904,0.035156,21728825.0,0.097273,1.730791e+09,320.0,166385,48.150146
166834,479234728.0,3.703076e-08,124.598565,42750.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.323123,2.503810e-07,0.000006,0.000149,7.562959,...,479234728.0,0.0,0.001589,0.022620,21728825.0,0.171962,1.730791e+09,320.0,174493,48.176637
167536,479234728.0,1.016800e-07,124.682014,43452.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.317695,2.013970e-06,0.000128,0.001258,7.728226,...,479234728.0,0.0,0.011572,0.060300,21728825.0,0.157299,1.730791e+09,320.0,173534,48.176198
169362,479234728.0,8.022791e-08,124.478415,45278.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.317919,1.988691e-06,0.000114,0.001160,9.418008,...,479234728.0,0.0,0.008901,0.048478,21728825.0,0.136189,1.730793e+09,320.0,171699,48.165194
170555,479234728.0,9.568017e-09,124.622276,46471.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.304680,8.969596e-08,0.000006,0.000274,3.833542,...,479234728.0,0.0,0.003891,0.035149,21728825.0,0.195838,1.730794e+09,320.0,175533,48.198644
172064,479234728.0,0.000000e+00,124.637748,47980.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.303783,4.282976e-08,0.000006,0.000241,4.638721,...,479234728.0,0.0,0.002997,0.016746,21728825.0,0.061123,1.730795e+09,320.0,158783,48.099244
172682,479234728.0,0.000000e+00,124.359202,48598.0,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",-0.328883,2.649933e-07,0.000001,0.000213,9.735327,...,479234728.0,0.0,0.004327,0.024982,21728825.0,0.062205,1.730795e+09,320.0,159052,48.076223


# Large FLOPS

In [6]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/9dbrt9iz').scan_history())  # VKDNW_EVOLUTION_FLOPS_1050M_17L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K7(64,80,2,64,3)SuperResIDWE1K5(80,160,2,64,1)SuperResIDWE2K7(160,160,2,144,3)SuperResIDWE4K3(160,112,2,128,1)SuperResIDWE6K5(112,88,1,104,4)SuperResIDWE6K7(88,280,1,256,3)SuperConvK1BNRELU(280,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
43718,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K7(64,...",1.038625e+09,398.0,43.363484,-0.238632,147.252831,-0.561208,54525,54.269899
51933,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K3(80,...",1.043663e+09,398.0,43.333759,-0.441092,150.402612,-0.519547,55053,54.249810
52790,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K7(64,...",1.038625e+09,398.0,43.302189,-0.247196,147.280646,-0.567733,55890,54.233330
35640,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K7(96,...",1.046856e+09,398.0,43.320169,-0.444891,149.679738,-0.515766,54347,54.223314
35905,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE4K5(64,...",1.046907e+09,398.0,43.312440,-0.248831,144.045527,-0.553383,54618,54.220559
43635,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE4K5(40,...",1.036969e+09,398.0,43.276116,-0.207543,143.860725,-0.555673,55391,54.198288
36471,"SuperConvK3BNRELU(3,240,2,1)SuperResIDWE1K7(24...",1.031410e+09,398.0,43.276861,-0.427323,148.686742,-0.491435,55030,54.192495
40517,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE4K5(64,...",1.049071e+09,398.0,43.279985,-0.278795,143.344023,-0.550783,53515,54.167702
54068,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K3(48,...",1.038267e+09,398.0,43.251806,-0.441185,149.685420,-0.541986,54884,54.164783
53085,"SuperConvK3BNRELU(3,240,2,1)SuperResIDWE1K7(24...",1.031410e+09,398.0,43.242878,-0.436239,148.712871,-0.504869,55205,54.161687


In [8]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/psxwtgax').scan_history())  # VKDNW_EVOLUTION_FLOPS_620M

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False)

'SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K3(80,64,2,48,2)SuperResIDWE1K3(64,128,2,72,1)SuperResIDWE2K5(128,160,2,176,2)SuperResIDWE4K5(160,136,2,144,3)SuperResIDWE6K7(136,168,1,168,4)SuperConvK1BNRELU(168,2048,1,1)' # VKDNW_EVOLUTION_FLOPS_620M

'SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,32,2,40,1)SuperResIDWE6K3(32,112,2,56,1)SuperResIDWE4K7(112,144,2,128,2)SuperResIDWE2K7(144,112,2,144,4)SuperResIDWE6K7(112,168,1,168,4)SuperConvK1BNRELU(168,2048,1,1)' # VKDNW_EVOLUTION_FLOPS_620M_2 (rank by 'vkdnw_combined' and take top

'SuperConvK3BNRELU(3,96,2,1)SuperResIDWE1K3(96,32,2,40,1)SuperResIDWE6K3(32,112,2,56,1)SuperResIDWE4K7(112,144,2,128,2)SuperResIDWE2K7(144,112,2,144,4)SuperResIDWE1K5(112,336,1,256,4)SuperConvK1BNRELU(336,2048,1,1)' # VKDNW_EVOLUTION_FLOPS_620M_3 (rank by 'rank' and take 2nd highest)

'SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K3(64,32,2,40,1)SuperResIDWE6K3(32,112,2,56,1)SuperResIDWE4K7(112,144,2,128,2)SuperResIDWE2K7(144,112,2,144,4)SuperResIDWE6K7(112,168,1,168,4)SuperConvK1BNRELU(168,2048,1,1)'

In [14]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/i48d6j79').scan_history())  # VKDNW_EVOLUTION_FLOPS_1050M_14L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
51893,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...",1.043056e+09,320.0,43.676054,-0.120016,124.625370,-0.654014,58437,54.651758
55087,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K5(96,...",1.045552e+09,320.0,43.655367,-0.119891,123.971445,-0.650525,59479,54.648745
52050,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K3(96,...",1.028293e+09,320.0,43.650443,-0.125385,125.289639,-0.649651,58893,54.633920
48037,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K3(96,...",1.028293e+09,320.0,43.660276,-0.126907,125.435997,-0.648024,57584,54.621276
51050,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K5(96,...",1.045552e+09,320.0,43.633551,-0.122600,123.953159,-0.652577,58596,54.611972
56532,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",1.037513e+09,320.0,43.606550,-0.126235,124.096159,-0.652091,59295,54.596830
54147,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K5(12...",1.031616e+09,320.0,43.571647,-0.125384,124.409081,-0.655698,60022,54.574114
58022,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K3(96,...",1.033510e+09,320.0,43.564665,-0.123662,123.997719,-0.654778,59923,54.565480
42365,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE2K3(80,...",1.033312e+09,320.0,43.579514,-0.122476,123.875468,-0.648821,59035,54.565399
38881,"SuperConvK3BNRELU(3,96,2,1)SuperResIDWE2K3(96,...",1.028293e+09,320.0,43.578011,-0.133403,125.319917,-0.655354,58912,54.561811


In [15]:
run.iloc[51893, :]

complexity                                              1043055648.0
vkdnw_lambda_1                                                   0.0
expressivity                                               124.62537
_step                                                        51893.0
arch               SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...
progressivity                                              -0.120016
vkdnw_lambda_2                                              0.000002
vkdnw_lambda_3                                              0.000102
vkdnw_lambda_4                                              0.001167
vkdnw_lambda_9                                              2.516253
vkdnw_lambda_10                                           101.409874
_runtime                                                64703.624751
vkdnw_lambda_7                                              0.092437
vkdnw_chisquare                                           167.345794
num_layers                        

In [5]:
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/kkphsati').scan_history())  # VKDNW_EVOLUTION_FLOPS_620M_17L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run[['vkdnw_dim', 'vkdnw_entropy']].apply(tuple, axis=1).rank(method='dense', ascending=True).astype(int)
run['az_nas'] = run['expressivity'].rank().apply(np.log)+run['progressivity'].rank().apply(np.log)+run['trainability'].rank().apply(np.log)+run['flops'].rank().apply(np.log)
run['vkdnw_combined'] = run['rank'].apply(np.log)+run['az_nas']

run[['arch', 'flops', 'vkdnw_dim', 'az_nas', 'progressivity', 'expressivity', 'trainability', 'rank', 'vkdnw_combined']].sort_values('vkdnw_combined', ascending=False).head(20)

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE2K5(48,32,2,24,1)SuperResIDWE1K3(32,88,2,56,1)SuperResIDWE6K3(88,128,2,120,2)SuperResIDWE4K5(128,136,2,144,3)SuperResIDWE6K7(136,168,1,168,4)SuperConvK1BNRELU(168,2048,1,1)'

,arch,flops,vkdnw_dim,az_nas,progressivity,expressivity,trainability,rank,vkdnw_combined
66760,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K7(24,...",614049574.0,370.0,44.146098,-0.348724,142.278717,-0.338584,58753,55.127195
65137,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",616740262.0,370.0,44.127790,-0.346623,138.650219,-0.331954,58703,55.108036
64724,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K7(24,...",614049574.0,370.0,44.149413,-0.347871,141.961359,-0.337790,57388,55.107004
49880,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE6K5(40,...",619912130.0,370.0,44.121343,-0.259295,135.078065,-0.325054,59015,55.106890
61484,"SuperConvK3BNRELU(3,64,2,1)SuperResIDWE1K7(64,...",614585046.0,370.0,44.211675,-0.342153,140.892607,-0.324678,53811,55.104909
61448,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",616740262.0,370.0,44.147541,-0.348585,138.659239,-0.327374,57362,55.104678
58256,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",616740262.0,370.0,44.120941,-0.352264,138.707169,-0.331892,58774,55.102395
57483,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE6K5(40,...",619912130.0,370.0,44.124312,-0.243530,135.249674,-0.329955,58556,55.102051
65163,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE6K5(40,...",619912130.0,370.0,44.120831,-0.258060,134.987327,-0.324248,58739,55.101690
59042,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",617202920.0,370.0,44.164558,-0.256727,136.032787,-0.306237,55892,55.095734


In [13]:
run.iloc[66760, :]

complexity                                               526635044.0
vkdnw_lambda_1                                                   0.0
expressivity                                              109.377612
_step                                                        66760.0
arch               SuperConvK3BNRELU(3,48,2,1)SuperResIDWE2K5(48,...
progressivity                                               -0.38408
vkdnw_lambda_2                                              0.000001
vkdnw_lambda_3                                              0.000004
vkdnw_lambda_4                                              0.000105
vkdnw_lambda_9                                              2.651457
vkdnw_lambda_10                                          1324.153198
_runtime                                                74757.318599
vkdnw_lambda_7                                              0.046403
vkdnw_chisquare                                           138.348624
num_layers                        

# Large model

In [15]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/nwcoszbb').scan_history()) # VKDNW_EVOLUTION_FLOPS_1050M_13L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run['vkdnw'].apply(np.log)+run['expressivity'].apply(np.log)+run['complexity'].apply(np.log)

run[['arch', 'model_size', 'vkdnw', 'expressivity', 'complexity', 'rank']].sort_values('rank', ascending=False).head(20)

'SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,72,2,56,1)SuperResIDWE1K7(72,80,2,128,1)SuperResIDWE4K3(80,80,2,144,1)SuperResIDWE1K7(80,432,2,576,4)SuperResIDWE1K5(432,576,1,680,4)SuperConvK1BNRELU(576,2048,1,1)' # VKDNW_1G_13L_a

,arch,model_size,vkdnw,expressivity,complexity,rank
84814,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",16788017.0,294.601236,126.464962,1.047843e+09,31.295587
97006,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",16705225.0,294.552422,126.315968,1.049110e+09,31.295451
94300,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",16705225.0,294.607602,126.262649,1.049110e+09,31.295216
87675,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE2K3(32,...",16257505.0,294.487850,126.429535,1.047921e+09,31.294997
95351,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",16788017.0,294.457866,126.427483,1.047843e+09,31.294804
96146,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",16705225.0,294.573362,126.171019,1.049110e+09,31.294374
97714,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",16705225.0,294.342611,126.269567,1.049110e+09,31.294371
92750,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",16788017.0,294.533882,126.320899,1.047843e+09,31.294219
92033,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",16705225.0,294.399506,126.223795,1.049110e+09,31.294202
97930,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...",16788017.0,294.523846,126.301908,1.047843e+09,31.294034


In [16]:
run.iloc[84814, :]

_runtime                                               109090.5537
_timestamp                                       1733268176.523888
num_layers                                                    13.0
flops                                                 1047842752.0
complexity                                            1047842752.0
model_size                                              16788017.0
arch             SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K3(32,...
vkdnw                                                   294.601236
expressivity                                            126.464962
trainability                                             -1.766521
_step                                                      84814.0
progressivity                                             -0.48079
rank                                                     31.295587
Name: 84814, dtype: object

In [10]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/rhl9g18y').scan_history()) # VKDNW_EVOLUTION_FLOPS_1050M_14L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run['vkdnw'].apply(np.log)+run['expressivity'].apply(np.log)+run['complexity'].apply(np.log)

run[['arch', 'model_size', 'vkdnw', 'expressivity', 'complexity', 'rank']].sort_values('rank', ascending=False).head(20)

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,40,2,72,1)SuperResIDWE1K7(40,96,2,256,1)SuperResIDWE1K5(96,496,2,264,1)SuperResIDWE2K7(496,360,2,368,4)SuperResIDWE1K5(360,576,1,256,5)SuperConvK1BNRELU(576,2048,1,1)' # VKDNW_1G_14L_a

'SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(120,48,2,72,1)SuperResIDWE1K3(48,96,2,320,1)SuperResIDWE1K5(96,400,2,216,3)SuperResIDWE1K3(400,432,2,352,5)SuperResIDWE1K5(432,368,1,128,2)SuperConvK1BNRELU(368,2048,1,1)' # VKDNW_1G_14L_b

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,120,2,80,1)SuperResIDWE1K5(120,144,2,208,1)SuperResIDWE2K7(144,160,2,144,3)SuperResIDWE2K3(160,360,2,352,2)SuperResIDWE1K7(360,576,1,312,5)SuperConvK1BNRELU(576,2048,1,1)' # VKDNW_1G_14L_c

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,48,2,72,1)SuperResIDWE1K3(48,128,2,208,1)SuperResIDWE2K7(128,160,2,144,3)SuperResIDWE1K5(160,432,2,576,5)SuperResIDWE1K3(432,720,1,400,2)SuperConvK1BNRELU(720,2048,1,1)' # VKDNW_1G_14L_d

,arch,model_size,vkdnw,expressivity,complexity,rank
99788,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12744729.0,320.463331,135.626954,1.046688e+09,31.448573
57103,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...",7953761.0,320.316986,134.579827,1.047965e+09,31.441585
87440,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",10419713.0,320.487585,134.198322,1.049611e+09,31.440848
78663,"SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...",9151977.0,320.311148,134.370906,1.047356e+09,31.439432
93841,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...",13298073.0,320.478987,135.153780,1.039659e+09,31.438388
74720,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12712969.0,320.562531,134.555874,1.043986e+09,31.438369
64864,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12712969.0,320.572913,134.524530,1.043986e+09,31.438169
98530,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12712969.0,320.546847,134.531806,1.043986e+09,31.438141
98907,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12712969.0,320.495871,134.542923,1.043986e+09,31.438065
77623,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...",12712969.0,320.442286,134.543267,1.043986e+09,31.437900


In [11]:
run.iloc[99788, :]

num_layers                                                    14.0
arch             SuperConvK3BNRELU(3,24,2,1)SuperResIDWE2K3(24,...
progressivity                                            -0.678166
_runtime                                             130969.108365
trainability                                             -1.410116
_step                                                      99788.0
vkdnw                                                   320.463331
model_size                                              12744729.0
flops                                                 1046688116.0
_timestamp                                        1733289491.60907
complexity                                            1046688116.0
expressivity                                            135.626954
rank                                                     31.448573
Name: 99788, dtype: object

In [12]:
run.iloc[57103, :]

num_layers                                                    14.0
arch             SuperConvK3BNRELU(3,120,2,1)SuperResIDWE1K7(12...
progressivity                                            -0.910551
_runtime                                              68672.874856
trainability                                             -1.159483
_step                                                      57103.0
vkdnw                                                   320.316986
model_size                                               7953761.0
flops                                                 1047965056.0
_timestamp                                       1733227195.375561
complexity                                            1047965056.0
expressivity                                            134.579827
rank                                                     31.441585
Name: 57103, dtype: object

In [13]:
run.iloc[87440, :]

num_layers                                                    14.0
arch             SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...
progressivity                                            -0.369198
_runtime                                             112495.338791
trainability                                             -1.191509
_step                                                      87440.0
vkdnw                                                   320.487585
model_size                                              10419713.0
flops                                                 1049611456.0
_timestamp                                       1733271017.839496
complexity                                            1049611456.0
expressivity                                            134.198322
rank                                                     31.440848
Name: 87440, dtype: object

In [14]:
run.iloc[93841, :]

num_layers                                                    14.0
arch             SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,...
progressivity                                            -0.313578
_runtime                                             122029.765173
trainability                                             -1.410788
_step                                                      93841.0
vkdnw                                                   320.478987
model_size                                              13298073.0
flops                                                 1039658576.0
_timestamp                                       1733280552.265878
complexity                                            1039658576.0
expressivity                                             135.15378
rank                                                     31.438388
Name: 93841, dtype: object

In [7]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/j9uelpj3').scan_history()) # VKDNW_EVOLUTION_FLOPS_1050M_16L

for col in run.columns:
    if col != 'arch':
        run[col] = run[col].astype('float')
        
run['rank'] = run['vkdnw'].apply(np.log)+run['expressivity'].apply(np.log)+run['complexity'].apply(np.log)

run[['arch', 'model_size', 'vkdnw', 'expressivity', 'complexity', 'rank']].sort_values('rank', ascending=False).head(20)

'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,72,2,48,1)SuperResIDWE1K7(72,80,2,128,1)SuperResIDWE2K5(80,320,2,112,1)SuperResIDWE1K5(320,576,2,352,3)SuperResIDWE1K5(576,576,1,352,3)SuperResIDWE1K5(576,464,1,520,5)SuperConvK1BNRELU(464,2048,1,1)' # VKDNW_1G_16L_a

'SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,48,2,72,1)SuperResIDWE1K7(48,80,2,128,1)SuperResIDWE2K7(80,160,2,144,3)SuperResIDWE1K5(160,360,2,576,4)SuperResIDWE1K7(360,464,1,520,5)SuperConvK1BNRELU(464,2048,1,1)' # VKDNW_1G_16L_b

,arch,model_size,vkdnw,expressivity,complexity,rank
95534,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",16549537.0,372.333517,159.167609,1.047778e+09,31.759686
99936,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.525611,156.554811,1.049713e+09,31.745495
95737,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.563077,156.470406,1.049713e+09,31.745056
98155,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.558857,156.468838,1.049713e+09,31.745035
99396,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.521104,156.482039,1.049713e+09,31.745018
81189,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.585310,156.450525,1.049713e+09,31.744988
99566,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.558706,156.455536,1.049713e+09,31.744949
99990,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.626033,156.426338,1.049713e+09,31.744943
96813,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.686159,156.393181,1.049713e+09,31.744893
92453,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...",14273665.0,372.550780,156.449763,1.049713e+09,31.744891


In [8]:
run.iloc[95534, :] # TOP 1 model

num_layers                                                    16.0
_step                                                      95534.0
_runtime                                             130491.922263
complexity                                            1047778464.0
trainability                                             -1.617353
model_size                                              16549537.0
expressivity                                            159.167609
arch             SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...
vkdnw                                                   372.333517
_timestamp                                       1733289577.889849
flops                                                 1047778464.0
progressivity                                             -0.37673
rank                                                     31.759686
Name: 95534, dtype: object

In [9]:
run.iloc[99936, :] # TOP 2 model (we also consider this one as the top 1 is very different than the others)

num_layers                                                    16.0
_step                                                      99936.0
_runtime                                             138505.842298
complexity                                            1049712984.0
trainability                                             -1.391101
model_size                                              14273665.0
expressivity                                            156.554811
arch             SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K7(72,...
vkdnw                                                   372.525611
_timestamp                                       1733297591.809884
flops                                                 1049712984.0
progressivity                                            -0.346809
rank                                                     31.745495
Name: 99936, dtype: object

# Nevergrad

In [2]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/szbj5dtw').scan_history()) # VKDNW_NEVERGRAD_WORKERS_L19_NgIohTuned_9335630

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,112,2,48,1)SuperResIDWE2K3(112,64,2,48,1)SuperResIDWE1K7(64,136,2,112,2)SuperResIDWE1K7(136,136,1,112,3)SuperResIDWE6K5(136,488,2,48,2)SuperResIDWE6K5(488,488,1,48,3)SuperResIDWE1K7(488,368,1,312,2)SuperResIDWE1K7(368,368,1,312,3)SuperConvK1BNRELU(368,2256,1,1)' # NEVERGRAD_NgIohTuned_L19_a

,net_str,model_size,num_layers,flops,score
14132,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,...",16568497.0,19.0,1.012042e+09,450.846179
21381,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,...",16568497.0,19.0,1.012042e+09,450.845749
37098,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,...",16568497.0,19.0,1.012042e+09,450.837001
53913,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",15429105.0,19.0,1.006023e+09,450.830032
68043,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",15429105.0,19.0,1.006023e+09,450.829936
64231,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",15429105.0,19.0,1.006023e+09,450.826728
49641,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,...",16568497.0,19.0,1.012042e+09,450.824530
68181,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",15429105.0,19.0,1.006023e+09,450.822679
47944,"SuperConvK3BNRELU(3,80,2,1)SuperResIDWE1K5(80,...",16568497.0,19.0,1.012042e+09,450.821755
60305,"SuperConvK3BNRELU(3,72,2,1)SuperResIDWE1K5(72,...",15429105.0,19.0,1.006023e+09,450.819961


In [7]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/xjwerpi9').scan_history()) # VKDNW_NEVERGRAD_L19_NgIohTuned_9335636

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,56,2,48,2)SuperResIDWE1K7(56,80,2,48,2)SuperResIDWE1K7(80,128,2,48,4)SuperResIDWE4K5(128,240,1,48,4)SuperResIDWE4K5(240,360,2,240,5)SuperConvK1BNRELU(360,2840,1,1)' # NEVERGRAD_NgIohTuned_L19_b

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K7(48,56,2,48,2)SuperResIDWE1K7(56,80,2,48,2)SuperResIDWE1K7(80,128,2,48,4)SuperResIDWE4K5(128,240,1,48,4)SuperResIDWE4K5(240,360,2,240,5)SuperConvK1BNRELU(360,2840,1,1)'

In [4]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/b7zetvas').scan_history()) # VKDNW_NEVERGRAD_WORKERS_L19_TwoPointsDE_9335629

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K7(24,136,2,48,1)SuperResIDWE2K7(136,24,2,96,1)SuperResIDWE1K7(24,120,2,72,2)SuperResIDWE1K7(120,120,1,72,5)SuperResIDWE6K5(120,264,2,104,2)SuperResIDWE6K7(264,368,1,88,2)SuperResIDWE6K7(368,368,1,88,4)SuperConvK1BNRELU(368,3120,1,1)' # NEVERGRAD_TwoPointsDE_L19_a

,net_str,model_size,num_layers,flops,score
36054,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K7(24,...",15236137.0,19.0,9.425579e+08,450.836520
27098,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",14269977.0,19.0,8.246329e+08,450.834348
80402,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",13045697.0,19.0,9.100138e+08,450.831217
77546,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",15656825.0,19.0,9.473122e+08,450.826743
44720,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",13376145.0,19.0,8.841165e+08,450.824806
69532,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",13512241.0,19.0,8.348737e+08,450.823365
44851,"SuperConvK3BNRELU(3,24,2,1)SuperResIDWE1K7(24,...",13813089.0,19.0,8.127482e+08,450.820367
60956,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K3(40,...",11948753.0,19.0,8.646245e+08,450.819518
35327,"SuperConvK3BNRELU(3,32,2,1)SuperResIDWE1K7(32,...",15677625.0,19.0,1.033486e+09,450.819137
35722,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K7(40,...",14997121.0,19.0,1.045999e+09,450.816846


In [1]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/ik7x5mai').scan_history()) # VKDNW_NEVERGRAD_L14_NgIohTuned_9336799

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,56,2,24,2)SuperResIDWE4K7(56,72,2,80,1)SuperResIDWE1K7(72,48,1,216,1)SuperResIDWE2K3(48,48,2,160,4)SuperResIDWE1K3(48,1008,2,168,4)SuperConvK1BNRELU(1008,3616,1,1)' # NEVERGRAD_NgIohTuned_L14_a

,net_str,model_size,num_layers,flops,score
80931,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.765879
26323,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.758681
21118,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.728512
43340,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.716021
3855,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.715969
7208,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.710341
4441,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.707878
45460,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.706950
88776,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.704793
51841,"SuperConvK3BNRELU(3,48,2,1)SuperResIDWE1K5(48,...",14584729.0,14.0,926537370.0,320.703640


In [5]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/1jfl59f2').scan_history()) # VKDNW_NEVERGRAD_L16_NgIohTuned_9336800

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K3(40,112,2,48,1)SuperResIDWE1K7(112,24,2,48,3)SuperResIDWE1K5(24,48,1,104,4)SuperResIDWE1K5(48,144,2,48,3)SuperResIDWE4K5(144,680,2,152,3)SuperConvK1BNRELU(680,2048,1,1)' # NEVERGRAD_NgIohTuned_L16_a

,net_str,model_size,num_layers,flops,score
20808,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K3(40,...",12640185.0,16.0,9.224670e+08,372.769459
70514,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10785993.0,16.0,9.105161e+08,372.742303
33313,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10380273.0,16.0,9.161410e+08,372.741106
45565,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10380273.0,16.0,9.161410e+08,372.736337
57750,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10785993.0,16.0,9.105161e+08,372.734134
22801,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10380273.0,16.0,8.309950e+08,372.731945
83679,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10785993.0,16.0,9.105161e+08,372.730469
13490,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K3(40,...",12696633.0,16.0,1.016374e+09,372.728935
43173,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K7(40,...",10380273.0,16.0,9.161410e+08,372.724830
16278,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE2K3(40,...",12640185.0,16.0,9.224670e+08,372.722865


In [9]:
import wandb 
import numpy as np
import pandas as pd
api_wandb = wandb.Api()
run = pd.DataFrame(api_wandb.run('nazderaze/VKDNW/c1gilnjb').scan_history()) # VKDNW_NEVERGRAD_L18_NgIohTuned_9336801

for col in run.columns:
    if col != 'net_str':
        run[col] = run[col].astype('float')
        
run[['net_str', 'model_size', 'num_layers', 'flops', 'score']].sort_values('score', ascending=False).head(20)

'SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,48,2,112,1)SuperResIDWE1K7(48,32,1,48,2)SuperResIDWE1K7(32,168,2,48,3)SuperResIDWE2K7(168,32,2,88,4)SuperResIDWE4K3(32,368,2,264,6)SuperConvK1BNRELU(368,1544,1,1)' # NEVERGRAD_NgIohTuned_L18_a

,net_str,model_size,num_layers,flops,score
29874,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.750613
40964,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,8.297656e+08,424.715133
45487,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,9.676387e+08,424.695069
35230,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.694056
28088,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.693039
20351,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",12992545.0,18.0,9.491923e+08,424.691104
48652,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.689376
41197,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.680485
42618,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,1.034398e+09,424.678346
41180,"SuperConvK3BNRELU(3,40,2,1)SuperResIDWE1K5(40,...",13099121.0,18.0,8.297656e+08,424.677972
